## Import Libraries

In [20]:
import os
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, start, listen  
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun



## Load the  LLM


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf", 
    n_threads=4
)


llama_model_load_from_file_impl: using device Metal (Apple M4) - 10922 MiB free
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimensio

In [16]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain.llms.base import LLM
from llama_cpp import Llama

class TinyLlamaLLM(LLM):
    model_path: str = Field(..., description="Path to GGUF model")
    
    def __init__(self, **data):
        super().__init__(**data)
        object.__setattr__(self, "_llm_instance", Llama(
            model_path=self.model_path,
            n_ctx=1024,
            n_threads=4,
        ))

    def _call(
        self, prompt: str, stop: Optional[list] = None, **kwargs
    ) -> str:
        result = self._llm_instance(prompt, stop=stop, max_tokens=256)
        return result["choices"][0]["text"]

    @property
    def _llm_type(self) -> str:
        return "llama-cpp"



In [17]:
llm = TinyLlamaLLM(model_path="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf")
output = llm.invoke("Explain LangChain in one line.")
print(output)


llama_model_load_from_file_impl: using device Metal (Apple M4) - 10916 MiB free
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimensio

 

LangChain is a language learning software that helps you learn a new language by providing interactive lessons, exercises, and quizzes. It uses the latest research in language acquisition and adaptive learning to optimize learning for each student's unique needs and pace.


## Load Tools

In [21]:
search_tool = DuckDuckGoSearchRun()
python_tool = PythonREPLTool()

## Agent Definitions



In [ ]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018-2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)
#This agent finds dataset links online.

In [ ]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[python_tool],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [ ]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[python_tool],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [ ]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [ ]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023). Focus on government sources. Print top 3 links.",
    agent=loader_agent
)

analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2–3 sentences.",
    agent=summarizer_agent
)
#Each task explains what the agent should do.

## Flow Definition

In [ ]:
flow = Sequential(
    steps=[
        load_task,
        Parallel(steps=[analyze_task, visualize_task, summarize_task])
    ]
)
#First, data is loaded. Then 3 tasks run in parallel.

##  Run and Show Result

In [ ]:
crew = Crew(flow=flow, verbose=True)
result = crew.kickoff()

print("\n Final Result:ssssss
